<a href="https://colab.research.google.com/github/Nithyarajoman/Machine-learning-Trial/blob/main/Effective_Data_Communication_Hierarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Sequential, Model

In [2]:
# Read data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')/255
x_train_2 = x_train.reshape(-1, 14, 14, 32).astype('float')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float')/255

print(x_train_2.shape)

4422102/4422102 [==============================] - 0s 0us/step
(7500, 14, 14, 32)


In [3]:
# Create Model
model_hierarchy = keras.models.Sequential(
    [        
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        
        models.Sequential(
            [
                layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(14, 14, 32),
                      strides=(2,2), padding="same",
                      use_bias=False),
                layers.BatchNormalization(),
                layers.Activation(activation="relu"),
            ]
        ),

        layers.Conv2D(64, kernel_size=(3, 3),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

In [ ]:
model_hierarchy.summary()

In [4]:
# Train
model_hierarchy.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_hierarchy.fit(x=x_train, y=y_train, epochs=5, batch_size=128, validation_split=0.1 )
score = model_hierarchy.evaluate(x_test, y_test, verbose=0)

Epoch 1/5
422/422 [==============================] - 23s 51ms/step - loss: 0.5907 - accuracy: 0.7914 - val_loss: 0.6763 - val_accuracy: 0.7800
Epoch 2/5
422/422 [==============================] - 21s 51ms/step - loss: 0.3898 - accuracy: 0.8587 - val_loss: 0.3194 - val_accuracy: 0.8833
Epoch 3/5
422/422 [==============================] - 22s 53ms/step - loss: 0.3412 - accuracy: 0.8752 - val_loss: 0.2953 - val_accuracy: 0.8943
Epoch 4/5
422/422 [==============================] - 22s 52ms/step - loss: 0.3142 - accuracy: 0.8857 - val_loss: 0.2765 - val_accuracy: 0.8988
Epoch 5/5
422/422 [==============================] - 22s 52ms/step - loss: 0.2934 - accuracy: 0.8929 - val_loss: 0.2810 - val_accuracy: 0.8980


In [5]:
# Building new model with multiple output layers
all_layers = [layer.output for layer in [model_hierarchy.get_layer(index=0),
                                        model_hierarchy.get_layer(index=1),
                                        model_hierarchy.get_layer(index=2),
                                        model_hierarchy.get_layer(index=3),
                                        model_hierarchy.get_layer(index=4),
                                        model_hierarchy.get_layer(index=5),
                                        model_hierarchy.get_layer(index=6),
                                        model_hierarchy.get_layer(index=7)]]


In [6]:
# Creating model

all_model = keras.Model(inputs=[model_hierarchy.inputs,model_hierarchy.get_layer('sequential').inputs], outputs=all_layers)
# all_model = keras.Model(inputs=[model_hierarchy.inputs,model_hierarchy.get_layer('sequential').get_input_at(0)], outputs=all_layers)
all_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv2d_input (InputLayer)      [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 32)   288         ['conv2d_input[0][0]']           
                                                                                                  
 batch_normalization (BatchNorm  (None, 14, 14, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 14, 14, 32)   0           ['batch_normalization[0][0]']

In [7]:
aux = all_model([x_train[:100],x_train_2[:100]])

In [37]:
# Function to calculate the number of non zer elements in each layer

def non_zeroes_layer(l):
    
    prob_non_zeroes=tf.math.count_nonzero(l)
    total=tf.size(l)

    prob_non_zeroes_count=prob_non_zeroes.numpy()
    total_count=total.numpy() 

    prob_non_zeroes_prob = prob_non_zeroes_count/total_count
    # print(prob_non_zeroes_prob) 
    return prob_non_zeroes_prob
    # zeroes = total.numpy() - y.numpy()
    # print("No of elements that are non-zeroes in  layer[]: %i " %prob_non_zeroes)

In [35]:
# Passing layers to the function to get the count

def summary_communication(model):
  print('%25s | %6s' % (
      'Layer Number', 'Effective communication'))
  print('*' * 100)

  for l,value in enumerate(model):
        x=model[l]
        probability = non_zeroes_layer(x)
        print('%25s | %5.4f' % (
                    l, probability ))
    
  return

In [38]:
summary_communication(aux)

             Layer Number | Effective communication
****************************************************************************************************
                        0 | 0.6448
                        1 | 1.0000
                        2 | 0.5019
                        3 | 0.5955
                        4 | 1.0000
                        5 | 1.0000
                        6 | 0.4427
                        7 | 0.4427
